# **Data processing**

## **Requirements**

In [3]:
import polars as pl
pl.Config(fmt_str_lengths=50)
pl.Config(tbl_rows=30)
import sys
import json
from unidecode import unidecode
import difflib
import pickle

In [4]:
sys.path.insert(0, r'C:\Users\fscielzo\Documents\Packages\BigEDA_Package_Private')
from BigEDA.preprocessing import dtypes_df, change_type, prop_cols_nulls
from BigEDA.descriptive import summary, outliers_table, freq_table, cross_quant_cat_summary, contingency_table_2D
from BigEDA.plots import histogram_matrix, boxplot_matrix, ecdfplot_matrix, barplot_matrix, boxplot_2D_matrix

In [5]:
def find_most_similar_string(target, candidates):
    # Get a list of matches with their similarity ratio
    matches = difflib.get_close_matches(target, candidates, n=1, cutoff=0.0)
    # Return the most similar string
    return matches[0] if matches else None

In [6]:
def has_dot(x):
    return x.find('.') != -1

def has_comma(x):
    return x.find(',') != -1

In [7]:
def reformat_str_number(x):

    # This functions is suppose to implement the following transformations:
    # '10554' --> '10554'
    # '10.554' --> '10554' (to be cast into integer or float)
    # '10.554.00' --> '10554' (to be cast into integer or float) 
    # '10,554' --> '10.554' (to be cast into float)
    # '10.554,75' --> '10554.75' (to be cast into float) 

    if has_comma(x):
        if not has_dot(x):
            return x.replace(',', '.')
        if has_dot(x):
            return ''.join(x.split(',')[0].split('.')) +'.'+ x.split(',')[1]
    elif has_dot(x) :
        if len(x.split('.')) > 2:
            return ''.join(x.split('.')[:2])
        else:
            return ''.join(x.split('.'))
    else: # number without ',' and '.'
        return  x

## **Conceptual description of the data**

Nombre del conjunto de datos: Accidentes de Tráfico de la Ciudad de Madrid 

Descripción: Accidentes de tráfico en la Ciudad de Madrid registrados por Policía Municipal 
con víctimas y/o daños al patrimonio. 

Unidad responsable: Dirección General de la Policía Municipal 
 
Descripción de estructura del fichero de la aplicación SIGIT 

IMPORTANTE: 

• **El fichero incluye un registro por persona implicada en el accidente (conductores, 
viajeros, peatones, testigos, etc.)**

• Los ficheros de 2010 a 2018 solo registran los accidentes con heridos o con daños al 
patrimonio municipal. 

• Los datos publicados son provisionales hasta seis meses después del año vencido. 

• Actualmente no se dispone de datos por barrio.

| Variable Name | Description | Type |
|----------|----------|----------|
| `num_expediente`    |  AAASNNNNNN, donde: • AAAA es el año del accidente. • S cuando se trata de un expediente con accidente.   |   String  |
| `fecha`    | Fecha en formato dd/mm/aaaa | Quantitative  |
| `hora`   |  La hora se establece en rangos horarios de 1 hora     |  Quantitative  |
| `localizacion`   | calle 1 ‐ calle 2 (cruce) o una calle   |  Quantitative  |
| `numero`   |   Número de la calle, cuando tiene sentido   |  Quantitative  |
| `cod_distrito`   |  Código del distrito    |  Multiclass  |
| `distrito`   |  Nombre del Distrito    |  Binary  |
| `tipo_accidente`   |  Colisión doble: Accidente de tráfico ocurrido entre dos vehículos en movimiento, (colisión frontal, fronto lateral, lateral) Colisión múltiple: Accidente de tráfico ocurrido entre más de dos vehículos en movimiento. Alcance: Accidente que se produce cuando un vehículo circulando o detenido por las circunstancias del tráfico es golpeado en su parte posterior por otro ehículo. Choque contra obstáculo o elemento de la vía: Accidente ocurrido entre un vehículo en movimiento con conductor y un objeto inmóvil que ocupa la vía o zona apartada de la misma, ya sea vehículo estacionado, árbol, farola, etc.  Atropello a persona: Accidente ocurrido ente un vehículo y un peatón que ocupa la calzada o que transita por aceras, refugios, paseos o zonas de la vía pública no destinada a la circulación de vehículos. Vuelco: Accidente sufrido por un vehículo con más de dos ruedas y que por alguna circunstancia sus neumáticos pierden el contacto con la calzada quedando apoyado sobre un costado o sobre el techo. Caída: Se agrupan todas las caídas relacionadas con el desarrollo y las circunstancias del tráfico, (motocicleta, ciclomotor, bicicleta, viajero bus, etc.,) Otras causas: Recoge los accidentes por atropello a animal, despeñamiento, salida de la vía, y otros |  Binary  |
| `estado_meteorologico`   |  Descripción climatología     |  Binary  |
| `tipo_vehiculo`   | tipo vehículo implicado   |  Multiclass  |
| `tipo_persona`   |  tipo persona implicada   |  Binary  |
| `rango_edad`   |  ramo edad persona afectada  |  Multiclass  |
| `sexo`   |  puede ser : hombre Mujer o no asignado   |  Multiclass  |
| `cod_lesividad`   |  01  Atención en urgencias sin posterior ingreso. - LEVE 02  Ingreso inferior o igual a 24 horas - LEVE 03 Ingreso superior a 24 horas. - GRAVE 04  Fallecido 24 horas - FALLECIDO 05  Asistencia sanitaria ambulatoria con posterioridad - LEVE 06  Asistencia sanitaria inmediata en centro de salud o mutua - LEVE 07  Asistencia sanitaria sólo en el lugar del accidente - LEVE 14  Sin asistencia sanitaria 77  Se desconoce En blanco Sin asistencia sanitaria  |  Multiclass  |
| `lesividad`   |  Descripción lesividad    |  Quantitative  |
| `coordenada_x_utm`   |  ubicación coordenada x     |  Quantitative  |
| `Coordenada_y_utm`   |  ubicación coordenada y    |  Quantitative  |
| `positiva_alcohol`   |  Puede ser N o S   |  Quantitative  |
| `positiva_droga`   |  Puede ser NULL o 1    |  Quantitative  |

## **Reading the data**

In [8]:
years = list(range(2019, 2025))
madrid_traffic_acc_df = {}
for year in years:
    madrid_traffic_acc_df[year] = pl.read_csv(fr'C:\Users\fscielzo\Documents\DataScience-GitHub\EDA\Madrid-Traffic-Accidents\data\{year}_Accidentalidad.csv', separator=';', ignore_errors=True)

In [9]:
madrid_traffic_acc_df[2019].head(4)

num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2018S017842""","""04/02/2019""","""9:10:00""","""CALL. ALBERTO AGUILERA, 1""","""1""",1,"""CENTRO""","""Colisión lateral""","""Despejado""","""Motocicleta > 125cc""","""Conductor""","""De 45 a 49 años""","""Hombre""","""7""","""Asistencia sanitaria sólo en el lugar del acciden…","""440068,049""","""4475679,17""","""N""","""NULL"""
"""2018S017842""","""04/02/2019""","""9:10:00""","""CALL. ALBERTO AGUILERA, 1""","""1""",1,"""CENTRO""","""Colisión lateral""","""Despejado""","""Turismo""","""Conductor""","""De 30 a 34 años""","""Mujer""","""7""","""Asistencia sanitaria sólo en el lugar del acciden…","""440068,049""","""4475679,17""","""N""","""NULL"""
"""2019S000001""","""01/01/2019""","""3:45:00""","""PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA…","""168""",11,"""CARABANCHEL""","""Alcance""","""NULL""","""Furgoneta""","""Conductor""","""De 40 a 44 años""","""Hombre""","""NULL""","""NULL""","""439139,603""","""4470836,854""","""S""","""NULL"""
"""2019S000001""","""01/01/2019""","""3:45:00""","""PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA…","""168""",11,"""CARABANCHEL""","""Alcance""","""NULL""","""Turismo""","""Conductor""","""De 40 a 44 años""","""Mujer""","""NULL""","""NULL""","""439139,603""","""4470836,854""","""N""","""NULL"""


In [10]:
madrid_traffic_acc_df[2019]

num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2018S017842""","""04/02/2019""","""9:10:00""","""CALL. ALBERTO AGUILERA, 1""","""1""",1,"""CENTRO""","""Colisión lateral""","""Despejado""","""Motocicleta > 125cc""","""Conductor""","""De 45 a 49 años""","""Hombre""","""7""","""Asistencia sanitaria sólo en el lugar del acciden…","""440068,049""","""4475679,17""","""N""","""NULL"""
"""2018S017842""","""04/02/2019""","""9:10:00""","""CALL. ALBERTO AGUILERA, 1""","""1""",1,"""CENTRO""","""Colisión lateral""","""Despejado""","""Turismo""","""Conductor""","""De 30 a 34 años""","""Mujer""","""7""","""Asistencia sanitaria sólo en el lugar del acciden…","""440068,049""","""4475679,17""","""N""","""NULL"""
"""2019S000001""","""01/01/2019""","""3:45:00""","""PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA…","""168""",11,"""CARABANCHEL""","""Alcance""","""NULL""","""Furgoneta""","""Conductor""","""De 40 a 44 años""","""Hombre""","""NULL""","""NULL""","""439139,603""","""4470836,854""","""S""","""NULL"""
"""2019S000001""","""01/01/2019""","""3:45:00""","""PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA…","""168""",11,"""CARABANCHEL""","""Alcance""","""NULL""","""Turismo""","""Conductor""","""De 40 a 44 años""","""Mujer""","""NULL""","""NULL""","""439139,603""","""4470836,854""","""N""","""NULL"""
"""2019S000001""","""01/01/2019""","""3:45:00""","""PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA…","""168""",11,"""CARABANCHEL""","""Alcance""","""NULL""","""Turismo""","""Conductor""","""De 45 a 49 años""","""Mujer""","""NULL""","""NULL""","""439139,603""","""4470836,854""","""N""","""NULL"""
"""2019S000001""","""01/01/2019""","""3:45:00""","""PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA…","""168""",11,"""CARABANCHEL""","""Alcance""","""NULL""","""Turismo""","""Pasajero""","""De 45 a 49 años""","""Mujer""","""NULL""","""NULL""","""439139,603""","""4470836,854""","""N""","""NULL"""
"""2019S000001""","""01/01/2019""","""3:45:00""","""PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA…","""168""",11,"""CARABANCHEL""","""Alcance""","""NULL""","""Turismo""","""Pasajero""","""De 65 a 69 años""","""Mujer""","""NULL""","""NULL""","""439139,603""","""4470836,854""","""N""","""NULL"""
"""2019S000001""","""01/01/2019""","""3:45:00""","""PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA…","""168""",11,"""CARABANCHEL""","""Alcance""","""NULL""","""Turismo""","""Pasajero""","""Más de 74 años""","""Mujer""","""NULL""","""NULL""","""439139,603""","""4470836,854""","""N""","""NULL"""
"""2019S000002""","""01/01/2019""","""3:50:00""","""CALL. CUART DE POBLET, 65""","""65""",10,"""LATINA""","""Choque contra obstáculo fijo""","""Despejado""","""Furgoneta""","""Conductor""","""Desconocido""","""Desconocido""","""NULL""","""NULL""","""436473,789""","""4472030,489""","""N""","""NULL"""


## **Removing columns**

In [11]:
madrid_traffic_acc_df[2022] = madrid_traffic_acc_df[2022].select(pl.all().exclude(['', '_duplicated_0']))

## **Replacing values**

In [12]:
replace_dict = {}
replace_dict['estado_meteorológico'] = {'NULL': None, 'Se desconoce': None}
replace_dict['rango_edad'] = {'Desconocido': None}
replace_dict['sexo'] = {'Desconocido': None}
replace_dict['cod_lesividad'] = {'NULL': None}
replace_dict['lesividad'] = {'NULL': None, 'Se desconoce': None}
replace_dict['positiva_alcohol'] = {'NULL': None, 'N': False, 'S': True}
replace_dict['positiva_droga'] = {'NULL': False, '1': True}
replace_dict['coordenada_x_utm'] = {'NULL': None, "#¡VALOR!": None}
replace_dict['coordenada_y_utm'] = {'NULL': None, "#¡VALOR!": None}
replace_dict['tipo_persona'] = {'NULL': None}
replace_dict['tipo_vehiculo'] = {'NULL': None, 'Sin especificar': None}
replace_dict['distrito'] = {'NULL': None}
replace_dict['tipo_accidente'] = {'NULL': None}


In [13]:
for year in years:
    for col in replace_dict.keys():
        try:
            madrid_traffic_acc_df[year] = madrid_traffic_acc_df[year].with_columns(pl.col(col).replace(replace_dict[col]).alias(col))
        except:
            pass

## **Changing data types**

In [14]:
columns_to_change = ['coordenada_x_utm', 'coordenada_y_utm']
for year in years:
    for col in columns_to_change:
        try:
            madrid_traffic_acc_df[year] = madrid_traffic_acc_df[year].with_columns(madrid_traffic_acc_df[year][col].map_elements(lambda x: x.replace(',', '.')).cast(pl.Float64).alias(col))
        except:
            pass

## **Data types**

In [15]:
for year in years:
    print(f'\n---------------------------------------\nData types of the columns of {year} df\n---------------------------------------')
    #display(dtypes_df(df=madrid_traffic_acc_df[year]))


---------------------------------------
Data types of the columns of 2019 df
---------------------------------------

---------------------------------------
Data types of the columns of 2020 df
---------------------------------------

---------------------------------------
Data types of the columns of 2021 df
---------------------------------------

---------------------------------------
Data types of the columns of 2022 df
---------------------------------------

---------------------------------------
Data types of the columns of 2023 df
---------------------------------------

---------------------------------------
Data types of the columns of 2024 df
---------------------------------------


## **Shapes**

In [16]:
for year in years:
    print(f'Shape of {year} df: {madrid_traffic_acc_df[year].shape}')

Shape of 2019 df: (51811, 19)
Shape of 2020 df: (32433, 19)
Shape of 2021 df: (41783, 19)
Shape of 2022 df: (47053, 19)
Shape of 2023 df: (48830, 19)
Shape of 2024 df: (11886, 19)


## **Columns**

In [17]:
for year in years:
    print(f'Columns of {year} df: {madrid_traffic_acc_df[year].columns}')

Columns of 2019 df: ['num_expediente', 'fecha', 'hora', 'localizacion', 'numero', 'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico', 'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad', 'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol', 'positiva_droga']
Columns of 2020 df: ['num_expediente', 'fecha', 'hora', 'localizacion', 'numero', 'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico', 'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad', 'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol', 'positiva_droga']
Columns of 2021 df: ['num_expediente', 'fecha', 'hora', 'localizacion', 'numero', 'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico', 'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad', 'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol', 'positiva_droga']
Columns of 2022 df: ['num_expediente', 'fecha', 'hora',

## **Unique values**

In [18]:
n_unique = {year: {} for year in years}

for year in years:
     print(f'\n------------------------------------------------------\nNumber of unique values for columns of {year} df\n------------------------------------------------------')
     for col in madrid_traffic_acc_df[year].columns :
          n_unique[year][col] = len(madrid_traffic_acc_df[year][col].unique())
          print(col, ':', n_unique[year][col])


------------------------------------------------------
Number of unique values for columns of 2019 df
------------------------------------------------------
num_expediente : 21934
fecha : 365
hora : 1214
localizacion : 15853
numero : 823
cod_distrito : 21
distrito : 21
tipo_accidente : 13
estado_meteorológico : 7
tipo_vehiculo : 30
tipo_persona : 3
rango_edad : 18
sexo : 3
cod_lesividad : 10
lesividad : 9
coordenada_x_utm : 21068
coordenada_y_utm : 21544
positiva_alcohol : 3
positiva_droga : 2

------------------------------------------------------
Number of unique values for columns of 2020 df
------------------------------------------------------
num_expediente : 14172
fecha : 366
hora : 1017
localizacion : 11069
numero : 707
cod_distrito : 22
distrito : 22
tipo_accidente : 13
estado_meteorológico : 7
tipo_vehiculo : 35
tipo_persona : 3
rango_edad : 18
sexo : 3
cod_lesividad : 10
lesividad : 9
coordenada_x_utm : 13829
coordenada_y_utm : 14037
positiva_alcohol : 3
positiva_droga : 2


In [19]:
'''
for year in years:
    print(f'\n--------------------------------------------------\nUnique values for columns of {year} df\n--------------------------------------------------')
    for col in madrid_traffic_acc_df[year].columns :
        display(madrid_traffic_acc_df[year][col].unique())
'''

"\nfor year in years:\n    print(f'\n--------------------------------------------------\nUnique values for columns of {year} df\n--------------------------------------------------')\n    for col in madrid_traffic_acc_df[year].columns :\n        display(madrid_traffic_acc_df[year][col].unique())\n"

In [20]:
#for col in madrid_traffic_acc_df[year].columns :
    #display(madrid_traffic_acc_df[year][col].unique())

## **Concatenating the data**

In [21]:
madrid_traffic_acc_df = pl.concat([madrid_traffic_acc_df[year] for year in years], how='vertical')

## **Replacing values (II)**

In [22]:
replace_dict = {}

replace_dict['rango_edad'] = {}
unique_values = madrid_traffic_acc_df['rango_edad'].unique().drop_nulls().to_list()
for i in range(len(unique_values)):
    limits = [x for x in unique_values[i].split(' ') if x.isdigit()]
    if len(limits) == 2:
        replace_dict['rango_edad'][unique_values[i]] = f'[{limits[0]},{limits[1]}]'
    elif len(limits) == 1:
        if 'Más' in unique_values[i].split(' '):
            replace_dict['rango_edad'][unique_values[i]] = f'> {limits[0]}'
        elif 'Menor' in unique_values[i].split(' '):
            replace_dict['rango_edad'][unique_values[i]] = f'< {int(limits[0])+1}'

unique_values = madrid_traffic_acc_df['tipo_vehiculo'].unique().drop_nulls().to_list()
replace_dict['tipo_vehiculo'] =  {x: 'Autobus' for x in unique_values if 'bus' in x or 'bús' in x}
replace_dict['tipo_vehiculo'].update({x: 'Moto' for x in unique_values if 'ciclo' in x.lower() or 'moto' in x.lower()})
replace_dict['tipo_vehiculo'].update({x: 'Maquinaria' for x in unique_values if 'maquinaria' in x.lower()})
replace_dict['tipo_vehiculo'].update({x: 'Patinete' for x in unique_values if 'patinete' in x.lower()})
replace_dict['tipo_vehiculo'].update({x: 'Remolque' for x in unique_values if 'remolque' in x.lower()})
replace_dict['tipo_vehiculo'].update({x: 'Otros' for x in unique_values if 'otros' in x.lower()})
replace_dict['tipo_vehiculo'].update({x: 'Bicicleta' for x in unique_values if 'bici' in x.lower()})

replace_dict['lesividad'] = {'Asistencia sanitaria inmediata en centro de salud o mutua': 'Asist. inmediata CS',
                             'Asistencia sanitaria ambulatoria con posterioridad': 'Asist. posterior CS',
                             'Ingreso inferior o igual a 24 horas': 'Ingreso <= 24h',
                             'Fallecido 24 horas': 'Fallecido',
                             'Atención en urgencias sin posterior ingreso': 'Urg. sin ingreso',
                             'Asistencia sanitaria sólo en el lugar del accidente': 'Asist. solo insitu',
                             'Ingreso superior a 24 horas': 'Ingreso > 24h',
                             'Sin asistencia sanitaria': 'Sin asist. sanitaria'
                             }

replace_dict['tipo_accidente'] = {'Solo salida de la vía': 'Salida via',
                                  'Atropello a animal': 'Atropello animal',
                                  'Atropello a persona': 'Atropello persona',
                                  'Choque contra obstáculo fijo': 'Choque obstaculo fijo'
                                  }

geojson_file = r'C:\Users\fscielzo\Documents\DataScience-GitHub\EDA\Madrid-Traffic-Accidents\data\madrid-districts.geojson'
with open(geojson_file) as f:
    madrid_geojson = json.load(f)
distritos_geojson = [madrid_geojson['features'][i]['properties']['name'] for i in range(len(madrid_geojson['features']))]
distritos_data = madrid_traffic_acc_df['distrito'].drop_nulls().unique().to_list()
replace_dict['distrito'] = {x: find_most_similar_string(unidecode(x.lower().title()), candidates=distritos_geojson) for x in distritos_data}

for col in replace_dict.keys():
    madrid_traffic_acc_df = madrid_traffic_acc_df.with_columns(pl.col(col).replace(replace_dict[col]).alias(col))

## Preprocessing districts data

In [23]:
district_data = {}

In [24]:
csv_name = 'indicadores_distritos_barrios'
distritos_barrios_df = pl.read_csv(fr'C:\Users\fscielzo\Documents\DataScience-GitHub\EDA\Madrid-Traffic-Accidents\data\{csv_name}.csv', 
                                      separator=';', ignore_errors=True)

distritos_barrios_df = distritos_barrios_df.select(['distrito', 'barrio', 'año', 'indicador_completo', 'valor_indicador'])\
                                           .drop_nulls(subset=['distrito'])\
                                           .pivot(
                                                index=["distrito", "barrio", "año"],
                                                columns="indicador_completo",
                                                values="valor_indicador",
                                                aggregate_function = 'first'
                                                )

distritos_barrios_df = distritos_barrios_df.select(['distrito',
                                                    'barrio',
                                                    'año',
                                                    'Superficie (Ha.) ',
                                                    'Población densidad (hab./Ha.) ',
                                                    'Número Habitantes',
                                                    'Población Hombres ',
                                                    'Población Mujeres',
                                                    'Edad media de la población',
                                                    'Población de 0 a 14 años   ',
                                                    'Población de 15 a 29 años',
                                                    'Población de 30 a 44  años',
                                                    'Población de 45 a 64 años',
                                                    'Población de 65 a 79 años      ',
                                                    'Población de 80 años y más           ',
                                                    'Población de 65 años y más          ',
                                                    'Personas con nacionalidad española             ',
                                                    'Personas con nacionalidad española Hombres                      ',
                                                    'Personas con nacionalidad española Mujeres                      ',
                                                    'Personas con nacionalidad extranjera            ',
                                                    'Personas con nacionalidad extranjera Hombres                       ',
                                                    'Personas con nacionalidad extranjera Mujeres                        ',
                                                    'Renta neta media anual de los hogares (Urban Audit)             ',
                                                    'Renta disponible media por persona',
                                                    'Paro registrado (número de personas registradas en SEPE en febrero) ',
                                                    'Paro registrado (número de personas registradas en SEPE en febrero) hombres    ',
                                                    'Paro registrado (número de personas registradas en SEPE en febrero) mujeres',
                                                    'Tasa absoluta de paro registrado (febrero)',
                                                    'Tasa absoluta de paro registrado Mujeres',
                                                    'Tasa de desempleo en mujeres de 16 a 24 años',
                                                    'Tasa de desempleo en mujeres de 25 a 44 años',
                                                    'Tasa de desempleo en mujeres de 45 a 64 años',
                                                    'Tasa absoluta de paro registrado Hombres ',
                                                    'Tasa de desempleo en hombres de 16 a 24 años',
                                                    'Tasa de desempleo en hombres de 25 a 44 años',
                                                    'Tasa de desempleo en hombres de 45 a 64 años',
                                                    'Personas paradas de larga duración (febrero)              ',
                                                    'Personas paradas de larga duración (febrero) hombres                         ',
                                                    'Personas paradas de larga duración (febrero) mujeres                            ',
                                                    'Personas paradas que SÍ perciben prestaciones (febrero)',
                                                    'Personas paradas que SÍ perciben prestaciones (febrero) hombres                         ',
                                                    'Personas paradas que SÍ perciben prestaciones (febrero) mujeres                         ',
                                                    'Personas paradas que NO perciben prestaciones (febrero)',
                                                    'Personas paradas que NO perciben prestaciones (febrero) hombres          ',
                                                    'Personas paradas que NO perciben prestaciones (febrero) mujeres          ',
                                                    'Población en etapas educativas',
                                                    'Alumnado en Centros privados concertados',
                                                    'Alumnado en Centros privados sin concierto',
                                                    'Alumnado en Centros públicos',
                                                    'Total alumnado extranjero ',
                                                    'Alumnado extranjero en Centros privados concertados                   ',
                                                    'Alumnado extranjero en Centros privados sin concierto                  ',
                                                    'Alumnado extranjero en Centros públicos                        ',
                                                    'Total alumnado con necesidades de apoyo educativo  ',
                                                    'Alumnado con necesidades de apoyo educativo en Centros privados concertados                   ',
                                                    'Alumnado con necesidades de apoyo educativo en Centros privados sin concierto                  ',
                                                    'Alumnado con necesidades de apoyo educativo en Centros públicos                        ',
                                                    'Población mayor/igual  de 25 años  que no sabe leer ni escribir o sin estudios                ',
                                                    'Población mayor/igual  de 25 años con enseñanza primaria incompleta  ',
                                                    'Población mayor/igual  de 25 años con Bachiller Elemental, Graduado Escolar, ESO, Formación profesional 1º grado  ',
                                                    'Población mayor/igual  de 25 años con Formación profesional 2º grado, Bachiller Superior o BUP  ',
                                                    'Población mayor/igual  de 25 años con titulación media, diplomatura, arquitectura o ingeniería técnica    ',
                                                    'Población mayor/igual  de 25 años  con estudios superiores, licenciatura, arquitectura, ingeniería sup., estudios sup. no universitarios, doctorado,  postgraduado  ',
                                                    'Población mayor/igual  de 25 años con Nivel de estudios desconocido y/o no consta                                   ',
                                                    'Casos trabajados por el programa de absentismo municipal',
                                                    'Número de personas con grado de discapacidad reconocido                               ',
                                                    'Número de personas con grado de discapacidad reconocido Hombres                                    ',
                                                    'Número de personas con grado de discapacidad reconocido Mujeres                                    ',
                                                    'Satisfacción de vivir en su barrio ',
                                                    'Calidad de vida actual en su barrio ',
                                                    'Percepción de seguridad en Madrid',
                                                    'Percepción de seguridad en el barrio por el día',
                                                    'Percepción de seguridad en el barrio por la noche',
                                                    'Solicitudes tramitadas de Renta Mínima de Inserción',
                                                    'Personas perceptoras de prestación de la Renta Mínima de Inserción',
                                                    'Personas beneficiarias de prestaciones sociales de carácter económico',
                                                    'Total prestaciones sociales de carácter económico ',
                                                    'Personas con Servicio de Ayuda a Domicilio (modalidad auxiliar de hogar)  ',
                                                    'Número Total de viviendas (Censo Edificios y Viviendas 2011)',
                                                    'Viviendas anteriores a 1980                                                                 ',
                                                    'Viviendas en estado ruinoso',
                                                    'Viviendas en estado malo ',
                                                    'Viviendas en estado deficiente  ',
                                                    'Viviendas en estado bueno ',
                                                    'Estado de la vivienda no consta ',
                                                    'Número Total de viviendas familiares (Censo Edificios y Viviendas 2011) ',
                                                    'Número Total de viviendas familiares (Censo Edificios y Viviendas 2011) Principal                           ',
                                                    'Número Total de viviendas familiares (Censo Edificios y Viviendas 2011)  Secundaria                           ',
                                                    'Número Total de viviendas familiares (Censo Edificios y Viviendas 2011)  Población desocupada    ',
                                                    'Intervenciones de la Policía Municipal en materia de seguridad: delitos relacionados con las personas',
                                                    'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia de armas',
                                                    'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con el patrimonio',
                                                    'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia y consumo de drogas',
                                                    'Atestados/partes de accidentes de tráfico confeccionados',
                                                    'Personas detenidas e investigadas por la Policía Municipal en materia de seguridad: Total  personas detenidas e investigadas ',
                                                    'Elecciones municipales, censo electoral',
                                                    'Elecciones municipales, abstenciones',
                                                    'Elecciones  municipales, votos en blanco',
                                                    'Elecciones municipales, votos a candidaturas',
                                                    'Elecciones municipales, votos a Más Madrid',
                                                    'Elecciones municipales, votos al Partido Popular',
                                                    'Elecciones municipales, votos a Ciudadanos-Partido de la Ciudadanía',
                                                    'Elecciones municipales, votos al Partido Socialista Obrero Español',
                                                    'Elecciones municipales, votos a VOX',
                                                    'Sedentarismo',
                                                    'Consumo de tabaco diario',
                                                    'Consumo de medicamentos ',
                                                    'Autopercepción de buen estado de salud  (porcentaje respuesta muy buena + buena)',
                                                    'Calidad de vida relacionada con la salud (CVRS) ',
                                                    'Presencia de enfermedad crónica ',
                                                    'Hipertensión arterial',
                                                    'Diabetes Mellitus',
                                                    'Personas con obesidad ',
                                                    'Personas con sobrepeso',
                                                    'Prevalencia de obesidad en la población infantil (2017. Estudio Madrid Salud. UCM)',
                                                    'Probabilidad de padecer enfermedad mental (GHQ-12)          (2018. EMS)',
                                                    'Grado de satisfacción con los espacios verdes',
                                                    'Grado de satisfacción con los parques infantiles ',
                                                    'Grado de satisfacción con los centros culturales',
                                                    'Grado de satisfacción con las instalaciones deportivas',
                                                    'Grado de satisfacción de los servicios sociales municipales',
                                                    'Superficie media construida (m2) inmuebles de uso residencial',
                                                    'Año medio de contrucción de inmuebles de uso residencial '])

columns_renamed = [x.lstrip(' ').rstrip(' ') for x in distritos_barrios_df.columns]
distritos_barrios_df = distritos_barrios_df.rename(dict(zip(distritos_barrios_df.columns, columns_renamed)))
distritos_df = distritos_barrios_df.filter(pl.col('barrio').is_null()).select(pl.exclude('barrio'))

columns_to_modify_type = [x for x in distritos_df.columns if x not in ['distrito', 'año']]
for col in columns_to_modify_type:
    distritos_df = distritos_df.with_columns(pl.col(col)\
                               .map_elements(lambda x: reformat_str_number(x))\
                               .cast(pl.Float64).round(2))

replace_dict = {x: find_most_similar_string(unidecode(x.lower().title()), candidates=distritos_geojson) for x in distritos_df['distrito'].drop_nulls().unique().to_list()}
distritos_df = distritos_df.with_columns(pl.col('distrito').replace(replace_dict).alias('distrito'))

columns_to_agg = [x for x in distritos_df.columns if x not in ['distrito', 'año']]
distritos_grouped_df = distritos_df.group_by(['distrito']).agg(pl.col(columns_to_agg).mean().round(2))
district_data[csv_name] = distritos_grouped_df


In [25]:
distritos_df.head(7)

distrito,año,Superficie (Ha.),Población densidad (hab./Ha.),Número Habitantes,Población Hombres,Población Mujeres,Edad media de la población,Población de 0 a 14 años,Población de 15 a 29 años,Población de 30 a 44 años,Población de 45 a 64 años,Población de 65 a 79 años,Población de 80 años y más,Población de 65 años y más,Personas con nacionalidad española,Personas con nacionalidad española Hombres,Personas con nacionalidad española Mujeres,Personas con nacionalidad extranjera,Personas con nacionalidad extranjera Hombres,Personas con nacionalidad extranjera Mujeres,Renta neta media anual de los hogares (Urban Audit),Renta disponible media por persona,Paro registrado (número de personas registradas en SEPE en febrero),Paro registrado (número de personas registradas en SEPE en febrero) hombres,Paro registrado (número de personas registradas en SEPE en febrero) mujeres,Tasa absoluta de paro registrado (febrero),Tasa absoluta de paro registrado Mujeres,Tasa de desempleo en mujeres de 16 a 24 años,Tasa de desempleo en mujeres de 25 a 44 años,Tasa de desempleo en mujeres de 45 a 64 años,Tasa absoluta de paro registrado Hombres,Tasa de desempleo en hombres de 16 a 24 años,Tasa de desempleo en hombres de 25 a 44 años,Tasa de desempleo en hombres de 45 a 64 años,Personas paradas de larga duración (febrero),Personas paradas de larga duración (febrero) hombres,…,Número Total de viviendas familiares (Censo Edificios y Viviendas 2011) Principal,Número Total de viviendas familiares (Censo Edificios y Viviendas 2011) Secundaria,Número Total de viviendas familiares (Censo Edificios y Viviendas 2011) Población desocupada,Intervenciones de la Policía Municipal en materia de seguridad: delitos relacionados con las personas,Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia de armas,Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con el patrimonio,Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia y consumo de drogas,Atestados/partes de accidentes de tráfico confeccionados,Personas detenidas e investigadas por la Policía Municipal en materia de seguridad: Total personas detenidas e investigadas,"Elecciones municipales, censo electoral","Elecciones municipales, abstenciones","Elecciones municipales, votos en blanco","Elecciones municipales, votos a candidaturas","Elecciones municipales, votos a Más Madrid","Elecciones municipales, votos al Partido Popular","Elecciones municipales, votos a Ciudadanos-Partido de la Ciudadanía","Elecciones municipales, votos al Partido Socialista Obrero Español","Elecciones municipales, votos a VOX",Sedentarismo,Consumo de tabaco diario,Consumo de medicamentos,Autopercepción de buen estado de salud (porcentaje respuesta muy buena + buena),Calidad de vida relacionada con la salud (CVRS),Presencia de enfermedad crónica,Hipertensión arterial,Diabetes Mellitus,Personas con obesidad,Personas con sobrepeso,Prevalencia de obesidad en la población infantil (2017. Estudio Madrid Salud. UCM),Probabilidad de padecer enfermedad mental (GHQ-12) (2018. EMS),Grado de satisfacción con los espacios verdes,Grado de satisfacción con los parques infantiles,Grado de satisfacción con los centros culturales,Grado de satisfacción con las instalaciones deportivas,Grado de satisfacción de los servicios sociales municipales,Superficie media construida (m2) inmuebles de uso residencial,Año medio de contrucción de inmuebles de uso residencial
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Centro""",2019,523.0,258.0,134881.0,67829.0,67052.0,44.03,10554.0,21800.0,42857.0,38025.0,14196.0,7449.0,21645.0,104041.0,51264.0,52777.0,30837.0,16564.0,14273.0,32458.0,24920.0,7595.0,3974.0,3621.0,7

In [26]:
distritos_grouped_df.head()

distrito,Superficie (Ha.),Población densidad (hab./Ha.),Número Habitantes,Población Hombres,Población Mujeres,Edad media de la población,Población de 0 a 14 años,Población de 15 a 29 años,Población de 30 a 44 años,Población de 45 a 64 años,Población de 65 a 79 años,Población de 80 años y más,Población de 65 años y más,Personas con nacionalidad española,Personas con nacionalidad española Hombres,Personas con nacionalidad española Mujeres,Personas con nacionalidad extranjera,Personas con nacionalidad extranjera Hombres,Personas con nacionalidad extranjera Mujeres,Renta neta media anual de los hogares (Urban Audit),Renta disponible media por persona,Paro registrado (número de personas registradas en SEPE en febrero),Paro registrado (número de personas registradas en SEPE en febrero) hombres,Paro registrado (número de personas registradas en SEPE en febrero) mujeres,Tasa absoluta de paro registrado (febrero),Tasa absoluta de paro registrado Mujeres,Tasa de desempleo en mujeres de 16 a 24 años,Tasa de desempleo en mujeres de 25 a 44 años,Tasa de desempleo en mujeres de 45 a 64 años,Tasa absoluta de paro registrado Hombres,Tasa de desempleo en hombres de 16 a 24 años,Tasa de desempleo en hombres de 25 a 44 años,Tasa de desempleo en hombres de 45 a 64 años,Personas paradas de larga duración (febrero),Personas paradas de larga duración (febrero) hombres,Personas paradas de larga duración (febrero) mujeres,…,Número Total de viviendas familiares (Censo Edificios y Viviendas 2011) Principal,Número Total de viviendas familiares (Censo Edificios y Viviendas 2011) Secundaria,Número Total de viviendas familiares (Censo Edificios y Viviendas 2011) Población desocupada,Intervenciones de la Policía Municipal en materia de seguridad: delitos relacionados con las personas,Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia de armas,Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con el patrimonio,Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia y consumo de drogas,Atestados/partes de accidentes de tráfico confeccionados,Personas detenidas e investigadas por la Policía Municipal en materia de seguridad: Total personas detenidas e investigadas,"Elecciones municipales, censo electoral","Elecciones municipales, abstenciones","Elecciones municipales, votos en blanco","Elecciones municipales, votos a candidaturas","Elecciones municipales, votos a Más Madrid","Elecciones municipales, votos al Partido Popular","Elecciones municipales, votos a Ciudadanos-Partido de la Ciudadanía","Elecciones municipales, votos al Partido Socialista Obrero Español","Elecciones municipales, votos a VOX",Sedentarismo,Consumo de tabaco diario,Consumo de medicamentos,Autopercepción de buen estado de salud (porcentaje respuesta muy buena + buena),Calidad de vida relacionada con la salud (CVRS),Presencia de enfermedad crónica,Hipertensión arterial,Diabetes Mellitus,Personas con obesidad,Personas con sobrepeso,Prevalencia de obesidad en la población infantil (2017. Estudio Madrid Salud. UCM),Probabilidad de padecer enfermedad mental (GHQ-12) (2018. EMS),Grado de satisfacción con los espacios verdes,Grado de satisfacción con los parques infantiles,Grado de satisfacción con los centros culturales,Grado de satisfacción con las instalaciones deportivas,Grado de satisfacción de los servicios sociales municipales,Superficie media construida (m2) inmuebles de uso residencial,Año medio de contrucción de inmuebles de uso residencial
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Arganzuela""",646.06,239.56,153691.0,71801.5,81889.5,44.85,17238.25,22836.5,37032.75,45653.0,21176.25,9754.25,30930.5,137948.0,64280.75,74269.5,16143.2

In [27]:
distritos_df['año'].unique()

año
i64
2008
2016
2017
2018
2019
2020
2021
2022
2023


In [41]:
csv_name_list = ['extranjeros_distritos', 'poblacion_distritos', 'intervenciones_policiales_distritos']

for csv_name in csv_name_list:
    
    district_data[csv_name] = pl.read_csv(fr'C:\Users\fscielzo\Documents\DataScience-GitHub\EDA\Madrid-Traffic-Accidents\data\{csv_name}.csv', 
                                            separator=';', ignore_errors=True)
    district_data[csv_name] = district_data[csv_name].rename({'Distrito': 'distrito'})
    district_data[csv_name] = district_data[csv_name].filter(~pl.col('distrito').is_in(['Ciudad de Madrid', 'Otras zonas']))\
                                                    .with_columns(pl.col('distrito').map_elements(lambda x: x.split('. ')[1]))
    replace_dict = {x: find_most_similar_string(unidecode(x.lower().title()), candidates=distritos_geojson) for x in district_data[csv_name]['distrito'].drop_nulls().unique().to_list()}
    district_data[csv_name] = district_data[csv_name].with_columns(pl.col('distrito').replace(replace_dict).alias('distrito'))

    if csv_name == 'poblacion_distritos':
        district_data[csv_name] = district_data[csv_name].select(pl.exclude('fecha recopilacion'))
    elif csv_name == 'intervenciones_policiales_distritos':
        district_data[csv_name] = district_data[csv_name].select(pl.exclude('Año'))
        district_data[csv_name] = district_data[csv_name].with_columns(pl.col('Intervenciones_Policiales_con_Detenidos').cast(str).str.strip_chars('.0'))
        district_data[csv_name] = district_data[csv_name].rename({'Intervenciones_Policiales_con_Detenidos': 'intervenciones_policiales_con_detenidos'})
    elif csv_name == 'extranjeros_distritos':
        district_data[csv_name] = district_data[csv_name]\
            .rename({'Total': 'poblacion', 'España': 'poblacion_española', 
                     'Otro país': 'poblacion_extranjera', '% de extranjeros': 'perc_poblacion_extranjera'})
        district_data[csv_name] = district_data[csv_name].with_columns(pl.col('poblacion_extranjera').cast(str))
    
    columns_to_modify_type = [x for x in district_data[csv_name].columns if x != 'distrito'] 
    for col in columns_to_modify_type:
        print(col)
        district_data[csv_name] = district_data[csv_name].with_columns(pl.col(col)\
                                                         .map_elements(lambda x: reformat_str_number(x))\
                                                         .cast(pl.Float64).round(2))    


poblacion
poblacion_española
poblacion_extranjera
perc_poblacion_extranjera
Total Población
% de 0 a 15 años
% de 16 a 64 años
% de 65 años y más
% de 80 años y más
Edad promedio
Crecimiento migratorio / Tasa de inmigración
intervenciones_policiales_con_detenidos


---

## **Saving the data**

In [459]:
madrid_traffic_acc_df.write_csv(fr'C:\Users\fscielzo\Documents\DataScience-GitHub\EDA\Madrid-Traffic-Accidents\data\madrid_traffic_accidents.csv')

In [42]:
with open(r'C:\Users\fscielzo\Documents\Streamlit-Apps\Madrid-Traffic-Accidents\data\district_data.pickle', 'wb') as file:
    pickle.dump(district_data, file)